In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# import scienceplots
# plt.style.use(['science','notebook','grid'])

plt.rcParams.update({
    # "lines.color": "white",
    # "patch.edgecolor": "white",
    # "text.color": "white",
    # "axes.facecolor": "#383838",
    # "axes.edgecolor": "lightgray",
    # "axes.labelcolor": "white",
    # "xtick.color": "white",
    # "ytick.color": "white",
    # "grid.color": "lightgray",
    # "figure.facecolor": "black",
    # "figure.edgecolor": "black",
    # "savefig.facecolor": "auto",
    # "savefig.edgecolor": "black",
	# "savefig.directory":savepath,
    "font.family":'Times New Roman',
	'figure.titleweight': 'bold',
	'figure.labelweight': 'bold',
	'figure.dpi':100})


In [74]:

s = 'Suelo	Magnitud	Profundidad	Distancia_km	area_integral	FFT_AmpMax	FFTFreq_AmpMax	FFT_power	FFT_std	FFT_q50	FFT_kurtosis	FFT_skew	Dist_hipocentro	PGA_Gal'
s = s.split()


In [75]:
from sklearn.preprocessing import TargetEncoder #type: ignore


data_og = pd.read_csv(r'C:\Users\huevo\Documents\GeofisicaFIUNAM\Tesis\DATOS\Datos_completo_Vs30.csv')#.dropna(axis=0)

data = data_og[(data_og['Calidad'] != 'X')]# & (data_og['Calidad'] != 'S/C')]

# area_integral FFT_AmpMax	FFTFreq_AmpMax	FFT_power	FFT_std	FFT_q50	FFT_kurtosis	FFT_skew
values = 'Suelo Vs30	Mw*	Profundidad	Distancia_km			Dist_hipocentro	PGA_Gal'.split()
data = data[values]
# data = data.drop(columns=['Clave','Lat_estacion', 'Lon_estacion', 'Fecha','Escala_magnitud','Lat_epicentro', 'Lon_epicentro', 'Calidad', 'Procedencia_datos', 'Fs_hz', 'Canales','Canal_1','Canal_2',	'Canal_3',	'Canal_4',	'Canal_5',	'Canal_6',	'Canal_7',	'Canal_8',	'Canal_9','Column1']) # ,'Unnamed: 0'
data = data.reindex(columns = [col for col in data.columns if col != 'PGA_Gal'] + ['PGA_Gal'])

data.to_csv(r'C:\Users\huevo\Documents\GeofisicaFIUNAM\Tesis\DATOS\Datos_entrenamiento_NC_Mw.csv', index=False)
# data_og.iloc[1135]

In [76]:

X = data.iloc[:,:-1]
y = data.iloc[:,-1]
te = TargetEncoder(smooth=100.0).fit(X,y)
target = TargetEncoder(smooth=100.0).fit_transform(X,y)
print(te.target_type_)
df = pd.DataFrame(np.c_[te.categories_[0], te.encodings_[0]], columns=['Categories','Encodings'])
df.to_csv(r'C:\Users\huevo\Documents\GeofisicaFIUNAM\Tesis\DATOS\Suelo_encoding_key.csv', index=False)

code = {}
for target, encoding in zip(te.categories_[0], te.encodings_[0]):
	code[target] = encoding

data['Suelo'] = data['Suelo'].replace(code)
data.to_csv(r'C:\Users\huevo\Documents\GeofisicaFIUNAM\Tesis\DATOS\Datos_entrenamiento_Mw.csv', index=False)
print(data.size)
data.head()


continuous
18963


C:\Users\huevo\AppData\Local\Temp\ipykernel_35232\3148439497.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Suelo'] = data['Suelo'].replace(code)


,Suelo,Vs30,Mw*,Profundidad,Distancia_km,Dist_hipocentro,PGA_Gal
0,44.276666,431.98129,5.01,50.0,27.436095,57.032791,230.226
1,44.276666,299.77307,6.93,84.0,229.794746,244.666355,59.241
2,44.276666,424.53152,6.93,84.0,137.512718,161.138907,199.404
3,44.276666,431.98129,6.93,84.0,414.740081,423.161122,26.432
4,44.276666,335.64410,6.93,84.0,215.051706,230.874936,17.628


In [77]:
from joblib import dump

dump(df, r'C:\Users\huevo\Documents\GeofisicaFIUNAM\Tesis\PROGRAMAS\Modelos\Modelos con Mw\Encodings.pkl')

['C:\\Users\\huevo\\Documents\\GeofisicaFIUNAM\\Tesis\\PROGRAMAS\\Modelos\\Modelos con Mw\\Encodings.pkl']

# Preprocesamiento

1. Seleccionar calidades diferentes de X
2. Codificar variables categoricas (Target Encoder)
3. Separar train_test
4. Estandarizar


## Get dummies

In [78]:
# Variables categoricas no necesarias para el entrenamiento del modelo (ya están correlacionadas entre sí con otras variables numéricas o no son relevantes)
# training_data = data.drop(columns=['Clave','Lat_estacion', 'Lon_estacion', 'Fecha','Lat_epicentro', 'Lon_epicentro', 'Calidad', 'Procedencia_datos', 'Fs_hz'])
# # 'area_integral','Fs_hz','FFT_AmpMax','FFTFreq_AmpMax','FFT_power','FFT_std','FFT_q50','FFT_kurtosis','FFT_skew'

# # Self-explanatory
# categorized = pd.get_dummies(training_data, columns=['Suelo'])

# # Cambia la columna objetivo al final
# categorized = categorized.reindex(columns = [col for col in categorized.columns if col != 'PGA_Gal'] + ['PGA_Gal']).drop(columns='Unnamed: 0')
# training_data = training_data.reindex(columns = [col for col in training_data.columns if col != 'PGA_Gal'] + ['PGA_Gal']).drop(columns='Unnamed: 0')

# categorized.to_csv(r'C:\Users\huevo\Documents\GeofisicaFIUNAM\Tesis\DATOS\Datos_entrenamiento.csv', index=False)
# training_data.to_csv(r'C:\Users\huevo\Documents\GeofisicaFIUNAM\Tesis\DATOS\Datos_entrenamiento_NC.csv', index=False)

In [79]:
# training_data.head()

In [80]:
# print(f'Tamaño del dataset: {categorized.size} elementos')
# print(categorized.shape)